# Assigment 5

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

In this path **..\_data\sbs\B_RawData\bancos** you will find scraped data from [this link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B). We get all the information of the last available day of every each month.

1. Generate a function called **import_data_sbs_banks** that take as arguments `start_month`, `start_year`, `end_month` and `end_year`. This function should be able to import the files in **..\_data\sbs\B_RawData\bancos** taking into consideration these arguments. Additionally, it should work if it is only given the first two arguments `(start_mont, start_year)`. The function should `returns` a nested dictionary. The `main key` must be year and `second key` must be `month`. The values should be the dataframes.

In [1]:
import os
import pandas as pd
from datetime import datetime

In [2]:
# Definir la funcion usando los 4 parametros indicados
def import_data_sbs_banks(start_month, start_year, end_month = None, end_year = None):

    # Leer los archivos usando el path y guardarlos en una lista
    path = r"../../_data\sbs\B_RawData\bancos"
    file_list = [os.path.join(path, file) for file in os.listdir(path)]
       
        # Crear un directorio vacio para guardar los dataframes
    excel_dict = {} 
    
    # Iterar cada elemento de file_list 
    for file in file_list:
        # Leer archivo excel y guardadr temporalmente
        df_tmp = pd.read_excel(file)
        # Crear columna date_info
        df_tmp["date_info"] = os.path.splitext(file)[0][45:]
        # Asignarle a la columna el formato de fecha
        df_tmp["date_info"] = pd.to_datetime(df_tmp["date_info"], format = "%d_%m_%Y")
        # Guardar en diccionario vacio creado previamente
        excel_dict[file] = df_tmp     
    
    # Condicional para que funcione incluso si introducimos 2 argumentos
    if end_month is None:
        end_month = start_month
    if end_year is None:
        end_year = start_year    
        
    # Crear fechas de inicio y fin mediante los argumentos start_date y end_date
    start_date = pd.to_datetime(f'{start_month}/01/{start_year}', format = '%m/%d/%Y')
    # Agregar condicion si es Diciembre para que tome hasta el primer dia del proximo ano como punto de corte
    # En caso diferente a diciembre, tomar todo el mes.

    if end_month == 12:
        end_date = pd.to_datetime(f'01/01/{end_year + 1}', format = '%m/%d/%Y')
    else: 
        end_date = pd.to_datetime(f'{end_month + 1}/01/{end_year}', format = '%m/%d/%Y') 

    # Seleccionar los Dataframes que esten entre las fechas especificadas
    selected_dfs = {
        k: v for k, v in excel_dict.items() 
        if start_date <= pd.Timestamp(v['date_info'].iloc[0]) <= end_date
    }
    
    # Ordenar los dataframes seleccionados de acuerdo a su fecha
    sorted_dfs = sorted(selected_dfs.items(), key =lambda x: (x[1]['date_info'].dt.year.values[0],
                                                              x[1]['date_info'].dt.month.values[0]))
    
    # Crear diccionario vacio para los nested dataframes.
    nested_dict = {}
    
    # Usamos 2 keys: mes y luego año de los dataframes ordenados
    # Especificar la estructura final del diccionario
    for key, value in sorted_dfs:
        year  = value['date_info'].dt.year.values[0]
        month = value['date_info'].dt.month.values[0]
        nested_dict.setdefault(year, {})[month] = value

    return nested_dict

In [3]:
#  Intento con 4 parámetros
import_data_sbs_banks(10, 2011, 12, 2011)
#Contenido de los dataframe en función del año y mes
#start_month: 10
#start_year: 2011
#end_month: 12
#end_year: 2011

{2011: {10:                                        Tasa Anual (%) Continental Comercio  \
  0                                        Corporativos        5.40        -   
  1                                          Descuentos        5.94        -   
  2                             Préstamos hasta 30 días        5.33        -   
  3                           Préstamos de 31 a 90 días        5.23        -   
  4                          Préstamos de 91 a 180 días        5.87        -   
  5                         Préstamos de 181 a 360 días        5.62        -   
  6                         Préstamos a más de 360 días        4.99        -   
  7                                    Grandes Empresas        7.50    12.02   
  8                                          Descuentos        7.50    15.54   
  9                             Préstamos hasta 30 días        6.97    10.91   
  10                          Préstamos de 31 a 90 días        7.61    12.00   
  11                         P

In [4]:
# Intento con 2 parámetros
import_data_sbs_banks(10, 2019)

#start_month: 10
#start_year: 2019
#end_month: Por defecto, start_month 
#end_year: Por defecto, start_year

{2019: {10:                                        Tasa Anual (%)   BBVA Comercio Crédito  \
  0                                        Corporativos   3.04     4.95    3.48   
  1                                          Descuentos   3.86        -    3.89   
  2                             Préstamos hasta 30 días   2.73        -    2.80   
  3                           Préstamos de 31 a 90 días   2.76     4.95    3.00   
  4                          Préstamos de 91 a 180 días   3.49        -    2.63   
  5                         Préstamos de 181 a 360 días   3.22        -    3.23   
  6                         Préstamos a más de 360 días   5.83        -    5.32   
  7                                    Grandes Empresas   6.69     7.22    5.50   
  8                                          Descuentos   9.55     5.51    5.06   
  9                             Préstamos hasta 30 días   4.14     7.50    5.16   
  10                          Préstamos de 31 a 90 días   6.12     7.81    5.